In [2]:
import sys, os, json, re
import numpy as np
import pandas as pd
from PIL import Image

In [3]:
im = Image.open('./printed.png')

In [4]:
im = im.convert('RGBA')

In [20]:
data = np.array(im)

In [10]:
data.shape

(293, 936, 4)

In [14]:
data[100][10]

array([214, 204, 190, 255], dtype=uint8)

In [25]:
red, green, blue, alpha = data.T

In [29]:
alpha.shape

(936, 293)

In [30]:
image_shape = alpha.shape

In [31]:
image_shape

(936, 293)

In [46]:
red[936][0]

IndexError: index 936 is out of bounds for axis 0 with size 936

In [42]:
np.tile(np.array([1,2,3,4,5]),3)

array([1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])

In [49]:
np.tile(range(image_shape[0]),2).shape

(1872,)

In [113]:
idf = pd.DataFrame(data = {
    'red': list(red.flatten()),
    'green': list(green.flatten()),
    'blue': list(blue.flatten()),
    'alpha': list(alpha.flatten()),
    'image_x_coord': np.tile(range(image_shape[0]),image_shape[1]),
    'image_y_coord': np.repeat(range(image_shape[1]),image_shape[0])
})

idf = idf.set_index(['image_x_coord','image_y_coord'])

- Background appears to be around ~ RGB(203,193,181)
- Ink appears to be around ~ RGB(91,94,132)

In [115]:
print('Range Red: {}'.format(203-91))

print('Range Green: {}'.format(193-94))

print('Range Blue: {}'.format(181-132))

Range Red: 112
Range Green: 99
Range Blue: 49


In [116]:
idf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 274248 entries, (0, 0) to (935, 292)
Data columns (total 4 columns):
red      274248 non-null int64
green    274248 non-null int64
blue     274248 non-null int64
alpha    274248 non-null int64
dtypes: int64(4)
memory usage: 9.4 MB


In [126]:
idf.describe()

,red,green,blue,alpha
count,274248.000000,274248.000000,274248.000000,274248.0
mean,196.959825,187.610499,179.377928,255.0
std,28.262669,25.496822,13.708768,0.0
min,62.000000,63.000000,103.000000,255.0
25%,199.000000,189.000000,178.000000,255.0
50%,204.000000,194.000000,182.000000,255.0
75%,209.000000,199.000000,186.000000,255.0
max,227.000000,215.000000,206.000000,255.0


In [117]:
from sklearn import cluster

In [118]:
# Convert DataFrame to matrix
mat = idf.values
# Using sklearn
km = cluster.KMeans(n_clusters=2)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
results = pd.DataFrame([idf.index,labels]).T

In [119]:
pd.Series(labels).value_counts()

0    256055
1     18193
dtype: int64

In [120]:
pd.Series(labels).shape

(274248,)

In [122]:
idf.loc[519].blue.sort_values(ascending=False).value_counts(sort=False)

117     1
120     1
122     1
123     2
124     1
131     2
133     1
134     1
135     1
136     1
142     1
143     2
145     1
160     2
166     1
169     1
171     1
172     1
174     6
175     8
176     9
177    13
178    13
179    15
180    21
181    23
182    17
183    27
184    19
185    16
186    26
187    11
188    11
189     7
190    11
191     7
192     7
193     2
194     1
195     1
Name: blue, dtype: int64

In [123]:
idf['background'] = np.array(np.abs(labels-1),dtype=bool)

In [128]:
idf.groupby('background').apply(lambda x: x.describe())

red          green           blue     alpha
background                                                             
False      count   18193.000000   18193.000000   18193.000000   18193.0
           mean       96.322871      97.075139     132.430165     255.0
           std        18.207035      16.427067      10.062709       0.0
           min        62.000000      63.000000     103.000000     255.0
           25%        84.000000      86.000000     126.000000     255.0
           50%        90.000000      91.000000     131.000000     255.0
           75%       103.000000     102.000000     137.000000     255.0
           max       153.000000     149.000000     173.000000     255.0
True       count  256055.000000  256055.000000  256055.000000  256055.0
           mean      204.110195     194.043139     182.713620     255.0
           std         7.826782       7.303615       5.134016       0.0
           min       144.000000     142.000000     149.000000     255.0
           25%       200.000000     190.000000     179.000000     255.0
           50%       205.000000     194.000000     183.000000     255.0
           75%       209.000000     199.000000     186.000000     255.0
           max       227.000000     215.000000     206.000000     255.0

In [130]:
idf.loc[idf.background==True,['red','green','blue']] = 255

In [136]:
idf.loc[:,['red','green','blue','alpha']].values

array([[255, 255, 255, 255],
       [255, 255, 255, 255],
       [255, 255, 255, 255],
       ...,
       [255, 255, 255, 255],
       [255, 255, 255, 255],
       [255, 255, 255, 255]])

In [146]:
new_red = idf.red.values.reshape(image_shape)
new_green = idf.green.values.reshape(image_shape)
new_blue = idf.blue.values.reshape(image_shape)
new_alpha = idf.alpha.values.reshape(image_shape)

In [147]:
new_image = np.dstack((new_red,new_green,new_blue,new_alpha))
new_image.shape

(936, 293, 4)

In [148]:

nim = Image.fromarray(new_image.astype('uint8'), 'RGBA')

In [149]:
nim.show()